# Homework2

## Weekly Retention Rate

### Anna Tigranyan

#### Libraries

In [13]:
import pandas as pd
import numpy as np
import datetime as dt

#### Importing data

In [14]:
df = pd.read_csv("online_retail_II.csv")

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [15]:
df = df[["InvoiceDate","Customer ID","Country"]]
df.head()

,InvoiceDate,Customer ID,Country
0,2009-12-01 07:45:00,13085.0,United Kingdom
1,2009-12-01 07:45:00,13085.0,United Kingdom
2,2009-12-01 07:45:00,13085.0,United Kingdom
3,2009-12-01 07:45:00,13085.0,United Kingdom
4,2009-12-01 07:45:00,13085.0,United Kingdom


In [16]:
df["InvoiceDate"] = pd.to_datetime(df['InvoiceDate']).dt.date

#### Country filter function

In [17]:
def get_country(data,column,country_name):
    return data.loc[data[column] == country_name]

# For example we wont to calculate monthly retention only for customers from "United Kingdom"  
df = get_country(df,"Country","United Kingdom")

df.head()

,InvoiceDate,Customer ID,Country
0,2009-12-01,13085.0,United Kingdom
1,2009-12-01,13085.0,United Kingdom
2,2009-12-01,13085.0,United Kingdom
3,2009-12-01,13085.0,United Kingdom
4,2009-12-01,13085.0,United Kingdom


In [18]:
def get_week(x): return dt.datetime(x.year, x.month, x.day) 

df['week'] = df['InvoiceDate'].apply(get_week) 

grouping = df.groupby('Customer ID')['week'] 

df['start_date'] = grouping.transform('min')

df.head()

,InvoiceDate,Customer ID,Country,week,start_date
0,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
1,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
2,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
3,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
4,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01


In [19]:
def get_int(df, column):
    year = df[column].dt.year
    week = df[column].dt.week 
    return year,week

year, week = get_int(df, 'week')

year_start,week_start = get_int(df, 'start_date')


<ipython-input-19-0d2b4354d2ca>:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  week = df[column].dt.week


In [20]:
years_diff = year - year_start   
week_diff = week - week_start 

df['Index'] = (week_diff+52*years_diff)
df.head()

,InvoiceDate,Customer ID,Country,week,start_date,Index
0,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
1,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
2,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
3,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
4,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0


In [21]:
grouping = df.groupby(['start_date', 'Index']) # Counting daily active user from each chort
data = grouping['Customer ID'].apply(pd.Series.nunique) # Counting number of unique customer Id's falling in each group
data = data.reset_index()
 # Assigning column names to the dataframe created above
counts = data.pivot(index='start_date',
                                 columns ='Index',
                                 values = 'Customer ID')
data.head()

,start_date,Index,Customer ID
0,2009-12-01,0.0,107
1,2009-12-01,1.0,30
2,2009-12-01,2.0,35
3,2009-12-01,3.0,5
4,2009-12-01,4.0,16


#### Weekly retention for UK

In [22]:
sizes = counts.iloc[:,0]
retention = counts.divide(sizes, axis=0)

retention.round(3)*100 # Coverting the retention rate into percentage 

Index,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,95.0,96.0,97.0,98.0,99.0,100.0,101.0,102.0,103.0,104.0
start_date,,,,,,,,,,,,,,,,,,,,,
2009-12-01,100.0,28.0,32.7,4.7,15.0,15.0,15.9,19.6,17.8,14.0,...,15.0,8.4,15.0,14.0,16.8,23.4,15.0,15.9,19.6,21.5
2009-12-02,100.0,27.0,34.8,7.9,12.4,12.4,18.0,21.3,11.2,10.1,...,13.5,16.9,9.0,20.2,13.5,21.3,19.1,16.9,22.5,19.1
2009-12-03,100.0,27.8,23.7,4.1,11.3,10.3,12.4,16.5,17.5,9.3,...,13.4,18.6,14.4,15.5,9.3,24.7,17.5,20.6,16.5,11.3
2009-12-04,100.0,30.3,27.3,10.6,15.2,13.6,9.1,16.7,18.2,15.2,...,6.1,7.6,1.5,15.2,21.2,12.1,19.7,10.6,16.7,22.7
2009-12-05,100.0,20.8,25.0,8.3,16.7,20.8,4.2,8.3,8.3,4.2,...,8.3,8.3,8.3,20.8,4.2,12.5,8.3,12.5,16.7,16.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-04,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-05,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-06,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
